In [ ]:
%pip install litellm reportlab python-dotenv

# Building an AI Agent

In this workshop, we will do the following:
- Define Agentic AI
- Build an AI agent using tools
- Understanding why agents are distributed systems
- Identify distributed system challenges for AI agents
- Recognize when agents become workflows

## What is an AI Agent?

An autonomous system that pursues goals through continuous decision-making and action

Think of an AI agent as an autonomous system that doesn't just respond once, but continuously works toward achieving a specific goal. Unlike traditional software that follows predetermined steps, an AI agent makes decisions dynamically based on the current situation and available information.

## Hands-on Moments

This is a hands-on workshop!

All of the instructors slides and code samples are are executable in the workshop notebooks.
We encourage you to follow along and play with the samples!

At the end of every chapter (notebook) will be a hands-on lab.
This a self-guided experience where the instructor gives a prompt (not an llm haha) with a notebook and some starter code and the attendees solve the puzzle.

We are going to create a Research Agent that makes a call to the OpenAI API, conducts research on a topic of your choice, and generates a PDF report from that research.

## Create a `.env` File

Next you'll create a `.env` file to store your API keys.
In the file browser on the left, create a new file and name it `.env`.

**Note**: It may disappear as soon as you create it. This is because Google Collab hides hidden files (files that start with a `.`) by default.
To make this file appear, click the icon that is a crossed out eye and hidden files will appear.

Then double click on the `.env` file and add the following line with your API key.

```
LLM_API_KEY = YOUR_API_KEY
LLM_MODEL = "openai/gpt-4o"
```

By default this notebook uses OpenAI's GPT-4o.
If you want to use a different LLM provider, look up the appropriate model name [in their documentation](https://docs.litellm.ai/docs/providers) and change the `LLM_MODEL` field and provide your API key.

In [ ]:
# Create .env file
with open(".env", "w") as fh:
  fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

# Now open the file and replace YOUR_API_KEY with your API key

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)


# Get LLM_API_KEY environment variable
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)

## Prompting the LLM

For it to be classified as an agent, it must prompt an LLM.

We use `litellm` here so you can use any LLM you please. All you need to do is provide an API key.

In [ ]:
from litellm import completion, ModelResponse


def llm_call(prompt: str, llm_api_key: str, llm_model: str) -> ModelResponse:
    response = completion(
      model=llm_model,
      api_key=llm_api_key,
      messages=[{ "content": prompt,"role": "user"}]
    )
    return response

# Change this to a fun prompt of your choice!
prompt = "Give me 5 fun Tardigrade facts in the form of a sea shanty."

result = llm_call(prompt, LLM_API_KEY, LLM_MODEL)["choices"][0]["message"]["content"]

print(result)

## Prompting the User

Now that we have our LLM call, we can write the code to prompt the user for their research topic.

In [ ]:
# Make the API call
print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ")
result = llm_call(prompt, LLM_API_KEY, LLM_MODEL)

# Extract the response content
response_content = result["choices"][0]["message"]["content"]

print("Research complete!")
print("-"*80)
print(response_content)


## Generating a PDF

Once you have your research data, you'll write it out to a PDF.
We'll use this later as a tool for our Agent to call.

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

def create_pdf(content: str, filename: str = "research_report.pdf") -> str:
    doc = SimpleDocTemplate(filename, pagesize=letter)

    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30,
        alignment=1
    )

    story = []
    title = Paragraph("Research Report", title_style)
    story.append(title)
    story.append(Spacer(1, 20))

    paragraphs = content.split('\n\n')
    for para in paragraphs:
        if para.strip():
            p = Paragraph(para.strip(), styles['Normal'])
            story.append(p)
            story.append(Spacer(1, 12))

    doc.build(story)
    return filename

create_pdf("Hello PDF!", filename="test.pdf")

## Open the PDF

Download the `test.pdf` PDF and open it. You should see a title **Research Report** and the words **Hello PDF!** in the document.

## Bringing it all together

You now have multiple functions that you can execute to achieve a task.
Next, write the code to bring this all together.

In [ ]:
# Make the API call
print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ")
result = llm_call(prompt, LLM_API_KEY, LLM_MODEL)

# Extract the response content
response_content: str = result["choices"][0]["message"]["content"]

pdf_filename = create_pdf(response_content, "research_report.pdf")
print(f"SUCCESS! PDF created: {pdf_filename}")


## The Foundations of an Agentic Application

You now have the foundations of an Agentic AI application.

What makes this agentic?
1. It's goal-oriented: it has a clear objective (generate a research report)
2. Tool usage: it combines multiple capabilities (LLM reasoning + PDF generation)
3. Autonomous decision making: the LLM decides how to structure and present the research

The functions you created are your "tools": `llm_call` and `create_pdf`. Some may think that an Agentic AI must have a loop, but that's not the case.

## Agentic Challenges

However, there are a few significant challenges to getting production AI at scale.

- Network resources (APIs and databases) go down
- Rate limitting
- LLMs are inherently non-deterministic



## Agents Don't Work in Isolation

* They calling other agents which call other agents, creating complex networks. Each agent has its own logic and potential failure points.
* Each of these nodes has its own
  * Event loop
  * Decision-making process

## Agents Call Other Agents

 Your research agent might:
  * Call a "Web Scraper" agent to gather sources
  * Call a "Fact Checker" agent to verify claims
  * Call a "Citation" agent to format references
  * Call a "PDF Generator" agent (what you built!)

## Your Simple Agent in Production

  What you built:
  - 1 LLM call
  - 1 PDF generation

  In production, this becomes:
  - Multiple LLM calls across different agents
  - External API calls (web scraping, databases)
  - File system operations
  - Network failures at any step
  - Need to coordinate responses from multiple agent

## Agents are Distributed Systems

<figure>
<center>
<img src='https://drive.google.com/uc?id=1yynE1_HDDVuFQjaesFcxyds045MzTkfo' />
<figcaption>The Truth About AI Agents</figcaption></center>
</figure>

## This is Why Agents == Workflows

  Your research agent is actually:
  1. Accept user input
    - Possible problems: input validation service, rate limitting
  2. Call the LLM for research
    - Possible problems: Internet connection, API down, rate limitting, timeout
  3. Generate PDF
    - Possible problems: Memory limits
  4. Return success/failure
    - Possible problem: Connection dropped

  Each step can fail.
  Each step might need different agents.
  This is a **workflow** - and workflows need orchestration

---
# Exercise 1 - Adding More Tools

* In these exercises you will:
  * **FILL IN**
* Go to the **Exercise** Directory in the Google Drive and open the **Practice** Directory
* Open _01-An-AI-Agent-Practice.ipynb_ and follow the instructions
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers
* **You have 5 mins**